<a href="https://colab.research.google.com/github/rst-analytics/rst-analytics2019/blob/master/KelRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Challenge RST Analytics 2019


## Défi KelRisk : Correction d'adresses + découverte d'adresses + éventuellement géocodage

Luc Mathis, en mode freestyle sur temps libre


### Etape 1 : Les données

* Fichier d'adresses BASIAS non géocodées et/ou mal formattées (fourni par la startup KelRisk)
* Liste des ICPE (fichier CSV), source = http://www.installationsclassees.developpement-durable.gouv.fr
* Liste des ICPE (fichier shape), source = georisque.gouv.fr
* Liste des établissements de la base IREP, année 2017, source = georisque.gouv.fr
* Base Sireine, retraité par C. Quest, version 1er mars 2019, http://data.cquest.org/geo_sirene/last




### Import des lib

In [0]:
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import re as re


!pip install Unidecode
from unidecode import unidecode

### Import des fichiers (situés sur Google Drive pour être traités via Colab)

Petit pré-traitement du fichier : Passage par OO, suppression des caractères " et export en csv avec séparateur ; plutôt que tabulation

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Chargement des données

In [0]:
# BASIAS KO
basias_ko_ini = pd.read_csv("/content/gdrive/My Drive/RST-Analytics/KelRisk/basias_adresse_KO-2.csv",sep=';')

In [0]:
basias_ko_df = basias_ko_ini.copy()
basias_ko_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
HNO2706512       4999 non-null object
Unnamed: 1       3372 non-null object
JEANDRE (STE)    4500 non-null object
27016            4997 non-null object
LES ANDELYS      4997 non-null object
Unnamed: 5       1910 non-null object
Unnamed: 6       1471 non-null object
dtypes: object(7)
memory usage: 273.5+ KB


On nomme les colonnes !

In [0]:
basias_ko_df.columns = ['id', 'type_activite','raison_sociale','code_commune',
                       'commune','adresse','commentaire']

basias_ko_df['adresse'] = basias_ko_df['adresse'].astype('str')


basias_ko_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
id                4999 non-null object
type_activite     3372 non-null object
raison_sociale    4500 non-null object
code_commune      4997 non-null object
commune           4997 non-null object
adresse           5000 non-null object
commentaire       1471 non-null object
dtypes: object(7)
memory usage: 273.5+ KB


In [0]:
basias_ko_df.head(10)

,id,type_activite,raison_sociale,code_commune,commune,adresse,commentaire
0,HNO2706514,NaN,VIDANGES BERNAYENNES LEGROS (STE DES),27016,LES ANDELYS,Terrain situé au Mont-Auger,NaN
1,IDF9200300,Blanchisserie,Blanchisserie Trajster S.A.,92007,BAGNEUX,"Fort-Grand, 7 place du",NaN
2,IDF9201016,Garage,Chenuil (Ets),92024,CLICHY,"Lorraine, 7 boulevard de",NaN
3,IDF9201030,Garage,Jaky (Ets),92044,LEVALLOIS-PERRET,"Lille, 4 rue de",NaN
4,IDF9201037,Dépôts de chiffons et papiers usés,Bohin (Etablissement),92004,ASNIERES-SUR-SEINE,"Fraternité, 6 avenue de la",NaN
5,IDF9201057,Atelier de fabrication de cartouches,Laguionie et Cie (Ets),92024,CLICHY,"Lorraine, 76/79 boulevard de",NaN
6,IDF9201060,Caisse d'emballage,Saget (Ets),92024,CLICHY,"Lorraine, 69 boulevard de",NaN
7,IDF9201757,Traitement de surface,RLB Roulier (Société) (SA),92073,SURESNES,"Billancourt, 89 rue de",NaN
8,HNO2705370,NaN,BARROISE (STE),27041,LA BARRE-EN-OUCHE,nan,NaN
9,HNO2705372,NaN,ZALKIN ET CIE,27414,MONTREUIL-L'ARGILLE,La Blardière,NaN


### On commence les regex dans la colonne adresse !

In [0]:
#On supprime les parenthèse dans le champ adresse pour homogénéiser
basias_ko_df["adresse"] =  basias_ko_df["adresse"].replace(regex=['\('], value='')
basias_ko_df["adresse"] =  basias_ko_df["adresse"].replace(regex=['\)'], value='')



In [0]:
# nouvelle colonne
basias_ko_df["adresse_regex"] = ""


In [0]:
basias_ko_df["adresse_regex"] =  basias_ko_df["adresse_regex"].replace(regex=['\,'], value='')

for i in range(0,len(basias_ko_df)):
  row = basias_ko_df.iloc[i]
  
  #print(row["adresse"])
  
  if row["adresse"] != np.nan : 
    #if re.search("^.*\,+ *([0-9])+.*\,*.*", row["adresse"]) is not None: 
      #cas classique Lorraine, 7 boulevard de
    #  address = re.split(r',', row["adresse"])
    #  # ex : ['Lorraine', ' 7 boulevard de']
    #  basias_ko_df["adresse_regex"].iloc[i] = address[1] + " " + address[0]
  
    if re.search("^.*([0-9])+.*", row["adresse"]) is not None:
      
      address = re.split(r'[0-9]+', row["adresse"])
      m = re.search(r"([0-9]+)", row["adresse"])
      num = m.group(0)
      
      basias_ko_df["adresse_regex"].iloc[i] = num + " " + address[1] + " " + address[0]
   
#suppr espaces en trop et les virgules  
  
basias_ko_df["adresse_regex"] =  basias_ko_df["adresse_regex"].replace(regex=['\,'], value='')  
basias_ko_df["adresse_regex"] =  basias_ko_df["adresse_regex"].replace(regex=['^ +'], value='') 
basias_ko_df.head(5000)   




      
      
      
   
  
  
  

,id,type_activite,raison_sociale,code_commune,commune,adresse,commentaire,adresse_regex
0,HNO2706514,NaN,VIDANGES BERNAYENNES LEGROS (STE DES),27016,LES ANDELYS,Terrain situé au Mont-Auger,NaN,
1,IDF9200300,Blanchisserie,Blanchisserie Trajster S.A.,92007,BAGNEUX,"Fort-Grand, 7 place du",NaN,7 place du Fort-Grand
2,IDF9201016,Garage,Chenuil (Ets),92024,CLICHY,"Lorraine, 7 boulevard de",NaN,7 boulevard de Lorraine
3,IDF9201030,Garage,Jaky (Ets),92044,LEVALLOIS-PERRET,"Lille, 4 rue de",NaN,4 rue de Lille
4,IDF9201037,Dépôts de chiffons et papiers usés,Bohin (Etablissement),92004,ASNIERES-SUR-SEINE,"Fraternité, 6 avenue de la",NaN,6 avenue de la Fraternité
5,IDF9201057,Atelier de fabrication de cartouches,Laguionie et Cie (Ets),92024,CLICHY,"Lorraine, 76/79 boulevard de",NaN,76 / Lorraine
6,IDF9201060,Caisse d'emballage,Saget (Ets),92024,CLICHY,"Lorraine, 69 boulevard de",NaN,69 boulevard de Lorraine
7,IDF9201757,Traitement de surface,RLB Roulier (Société) (SA),92073,SURESNES,"Billancourt, 89 rue de",NaN,89 rue de Billancourt
8,HNO2705370,NaN,BARROISE (STE),27041,LA BARRE-EN-OUCHE,nan,NaN,
9,HNO2705372,NaN,ZALKIN ET CIE,27414,MONTREUIL-L'ARGILLE,La Blardière,NaN,


In [0]:
basias_ko_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
id                4999 non-null object
type_activite     3372 non-null object
raison_sociale    4500 non-null object
code_commune      4997 non-null object
commune           4997 non-null object
adresse           5000 non-null object
commentaire       1471 non-null object
adresse_regex     5000 non-null object
dtypes: object(8)
memory usage: 312.6+ KB


In [0]:
basias_ko_df.to_csv("/content/gdrive/My Drive/RST-Analytics/KelRisk/basias_ko_regex.csv")

In [0]:
print(re.search("^.*([0-9])+.*", "Gare, 3, Place de la"))

address = re.split(r'[0-9]+', "Gare, 3, Place de la")

m = re.search(r"([0-9]+)", "Gare, 3, Place de la")

print(address)
print(m.group(0))

address = re.split(r',', "Gare, 3, Place de la")
print(address)

#virer les virgules


<_sre.SRE_Match object; span=(0, 20), match='Gare, 3, Place de la'>
['Gare, ', ', Place de la']
3
['Gare', ' 3', ' Place de la']
